In [1]:
import zipfile
import os
import pandas as pd
import numpy as np
import random
import string
import openpyxl
import pdb


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\berce\programming\Management\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [13]:
#cycle though .xlsx files in zip_ref

dir_path = r'C:\Users\berce\programming\Tiszaujvaros'

sheets_dict = {}


#files from dir_path and subdirs
for root, dirs, files in os.walk(dir_path):
    for file in files:
        if not file.endswith('.xlsx'):
            continue

        #read in file
        xl_file = pd.read_excel(os.path.join(root,file), sheet_name=None)
        
        osszegzes = xl_file['Osszegzes'].transpose().reset_index()
        osszegzes.columns = osszegzes.iloc[0]
        osszegzes = osszegzes.iloc[1:]
        print(osszegzes)

        # concatenate the sheets into a single dataframe for each sheet
        for sheet_name, sheet_df in xl_file.items():
            if sheet_name == 'Osszegzes':
                continue
            #replace ' ' with NaN
            sheet_df.replace(' ', np.nan, inplace=True)
            sheet_df.replace('', np.nan, inplace=True)
            sheet_df.dropna(how='all', inplace=True)        
            #get filename without extension
            #print(osszegzes['Izometria'].values[0])
            sheet_df['Lap'] = osszegzes['Lap'].values[0]

            sheet_df['Izometria'] = osszegzes['Izometria'].values[0]

            sheet_df['ID'] = sheet_df['Izometria'].apply(lambda x: ''.join(random.choices(string.ascii_lowercase + string.digits, k=8)))

            sheet_df['Típus'] = sheet_df['Típus'].apply(lambda x: x.capitalize() if isinstance(x, str) else x)

            #capitalize all columns
            sheet_df.columns = sheet_df.columns.str.capitalize()
            
            if sheet_name in sheets_dict:
                sheets_dict[sheet_name] = pd.concat([sheets_dict[sheet_name], sheet_df])
            else:
                sheets_dict[sheet_name] = sheet_df

            
print(sheets_dict['Varrat'])             
print(sheets_dict['Varrat'])





0                Izometria Lap Anyagminoseg Magassag (m)       Alvállalkozó
1  P200007-100-EMPDIO-0005   1       TP304L           10  Privát Technoszer
0                Izometria Lap Anyagminoseg Magassag (m)       Alvállalkozó
1  P200007-100-EMPDIO-0005  10       TP304L            3  Privát Technoszer
0                Izometria Lap Anyagminoseg Magassag (m)       Alvállalkozó
1  P200007-100-EMPDIO-0005  11       TP304L            3  Privát Technoszer
0                Izometria Lap Anyagminoseg Magassag (m)       Alvállalkozó
1  P200007-100-EMPDIO-0005  12       TP304L            3  Privát Technoszer
0                Izometria Lap Anyagminoseg Magassag (m)       Alvállalkozó
1  P200007-100-EMPDIO-0005  13       TP304L            3  Privát Technoszer
0                Izometria Lap Anyagminoseg Magassag (m)       Alvállalkozó
1  P200007-100-EMPDIO-0005  14       TP304L            3  Privát Technoszer
0                Izometria Lap Anyagminoseg Magassag (m)       Alvállalkozó
1  P200007-1

In [17]:
sheet_df.head()

,Sorszám,Típus,Dn,Pn,Unnamed: 4,Lapszam,Izometria,Random
0,K1,Pn10-40,300,25,NaN,1,P200007-100-EMPDIO-0045,jsuexif0
1,K2,Pn10-40,300,25,NaN,1,P200007-100-EMPDIO-0045,tfyooyp9


In [22]:
# save each concatenated sheet to a separate CSV file
for sheet_name, sheet_df in sheets_dict.items():


    sheet_df['Lapszam'] = sheet_df['Lapszam'].astype(int)
    #read in Torzs
    izometrialapok = pd.read_excel(r'C:\Users\berce\Planning\Tasks\Torzs.xlsx', sheet_name='Izometrialap')

    #left join on Izometria and lapszam

    sheet_df = sheet_df.merge(izometrialapok, how='left', left_on=['Izometria', 'Lapszam'], right_on=['Izometria', 'Lap'])

    csv_file_name = sheet_name + '.csv'
    sheet_df.to_csv(csv_file_name, index=False, encoding='windows-1250')
    print(f'{csv_file_name} saved to disk')

Csőtartó.csv saved to disk
Cső.csv saved to disk
Varrat.csv saved to disk
Karima.csv saved to disk
